## Logistic Regression 多分类实战（MNIST）

此处 Loss function 采用 crossEntropyLoss

In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

In [2]:
from visdom import Visdom

In [3]:
batch_size=200 #Batch Size：一次训练所选取的样本数
learning_rate=0.01
epochs=3 #1个epoch表示过1遍训练集中的所有样本，这里可以设置为 5

#加载数据
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data/', train=False, download=True,
                   transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

In [3]:
# 在pytorch中的定义（a，b）a是out输出，b是in输入，也就是（输出，输入）
# 比如第一个可以理解为从784降维成200的层
w1, b1 = torch.randn(200, 784, requires_grad=True),\
         torch.zeros(200, requires_grad=True)
w2, b2 = torch.randn(200, 200, requires_grad=True),\
         torch.zeros(200, requires_grad=True)
w3, b3 = torch.randn(10, 200, requires_grad=True),\
         torch.zeros(10, requires_grad=True)

#凯明初始化
# torch.nn.init.kaiming_normal_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu')
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)

tensor([[ 0.1054, -0.0533, -0.0245,  ..., -0.1081,  0.0521,  0.0839],
        [ 0.0523, -0.0468,  0.0551,  ..., -0.0937, -0.0934,  0.0383],
        [-0.0553, -0.0256,  0.0705,  ..., -0.0381,  0.1084, -0.0544],
        ...,
        [ 0.1206, -0.0403, -0.0034,  ..., -0.0679,  0.0092, -0.0966],
        [ 0.1148,  0.0669, -0.0461,  ..., -0.1810, -0.1067,  0.0905],
        [ 0.1352, -0.0624,  0.0797,  ..., -0.0476,  0.1126, -0.0556]],
       requires_grad=True)

In [4]:
#前向传播过程
def forward(x):
    x = x@w1.t() + b1
    x = F.relu(x)
    x = x@w2.t() + b2
    x = F.relu(x)
    x = x@w3.t() + b3
    x = F.relu(x)  #这里不要用softmax，因为之后的crossEntropyLoss中自带了。这里可以用relu，也可以不用。
    return x  #返回的是一个logits（即没有经过sigmoid或者softmax的层）

In [5]:
#优化器
optimizer = optim.SGD([w1, b1, w2, b2, w3, b3], lr=learning_rate)
criteon = nn.CrossEntropyLoss()

In [6]:
for epoch in range(epochs):
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28) #将二维的图片数据打平 [200,784],第5课用的 x = x.view(x.size(0), 28*28)

        logits = forward(data) #这里是网络的输出
        loss = criteon(logits, target)  #调用cross—entorpy计算输出值和真实值之间的loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
        # len(data)---指的是一个batch_size;
        # len(train_loader.dataset)----指的是train_loader这个数据集中总共有多少张图片（数据）
        # len(train_loader)---- len(train_loader.dataset)/len(data)---就是这个train_loader要加载多少次batch

    # 测试网络---test----每训练完一个epoch检测一下测试结果
    # 因为每一个epoch已经优化了batch次参数，得到的参数信息还是OK的
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        logits = forward(data) #logits的shape=[200,10]，--200是batchsize，10是最后输出结果的10分类
        test_loss += criteon(logits, target).item()  #每次将test_loss进行累加   #target=[200,1]---每个类只有一个正确结果

        pred = logits.data.max(1)[1]
        # 这里losgits.data是一个二维数组；其dim=1;max()---返回的是每行的最大值和最大值对应的索引
        # max(1)----是指每行取最大值;max(1)[1]---取每行最大值对应的索引号
        # 也可以写成 pred=logits.argmax(dim=1)
        correct += pred.eq(target.data).sum()
        #预测值和目标值相等个数进行求和--在for中，将这个test_loader中相等的个数都求出来
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.622670
Train Epoch: 0 [20000/60000 (33%)]	Loss: 1.169736
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.597393

Test set: Average loss: 0.0030, Accuracy: 8096/10000 (81%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.600992
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.528212
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.460533

Test set: Average loss: 0.0025, Accuracy: 8305/10000 (83%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.506943
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.498760
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.455540

Test set: Average loss: 0.0023, Accuracy: 8401/10000 (84%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.466507
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.545814
Train Epoch: 3 [40000/60000 (67%)]	Loss: 0.493273

Test set: Average loss: 0.0022, Accuracy: 8464/10000 (85%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.391793
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.471149
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.385770

Test set: Average loss: 0.0021

### 模型改进：交叉验证+GPU加速+可视化+正则化+动量+学习率衰减

设置交叉验证，即 训练集：验证集=5:1

In [3]:
batch_size=200
learning_rate=0.01
epochs=3

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data/', train=False, download=True,
                   transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)# 训练集方法不变

train_db = datasets.MNIST('data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
train_db, val_db = torch.utils.data.random_split(train_db, [50000, 10000])# 训练集划分两份

train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_db,
    batch_size=batch_size, shuffle=True)

这一部分我们尝试用另一种方法构建全连接层

In [4]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            # nn.Dropout(0.5), dropout 50%
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )
        #一般采用以下的写法，nn.Linear自带了自己的初始化方式，一般够用

    def forward(self, x):
        x = self.model(x)

        return x

并将设备与数据都搬入GPU,同时设置正则化参数

In [5]:
device = torch.device('cuda:0')
net = MLP().to(device)
criteon = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.01)
# 这里设置 momentum=0.9,新增了动量。有些优化器没有 momentum参数，因为优化器本身带有动量影响。
# weight_decay=0.01，意思是L2-regularization，lamda设置为0.01
scheduler=torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                         factor=0.1, patience=10, verbose=False, 
                                         threshold=0.0001, threshold_mode='rel', 
                                         cooldown=0, min_lr=0, eps=1e-08)
#训练过程中，optimizer会把 learning rate 交给scheduler管理
#当指标（比如loss）连续patience次数还没有改进时，需要降低学习率，factor为每次下降的比例

#或者可以采用 scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=30,gamma=0.1) 每30步将 lr 乘 0.1

可以加入 Visdom 可视化功能

In [6]:
viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='train loss'))
viz.line([[0.0, 0.0]], [0.], win='test_loss', opts=dict(title='test loss&acc.',legend=['loss', 'acc.']))
global_step = 0

Setting up a new session...


In [7]:
for epoch in range(epochs):
    #用train_loader来训练
    # net_dropped.train() 启用dropout
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)
        schedular.step(loss) # 检查 loss 的减少有没有长时间静止
        
        #pytorch里没有自动的L1-regularization
        #如果自己写代码将如下：
        """
        regularization_loss=0
        for param in model.parameters():
        regularization_loss += torch.sum(torch.abs(param))
        
        classify_loss=criteon(logits, target)
        loss=classify_loss+0.01*regularization_loss
        """

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        global_step += 1
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

    #用 val_loader来测试，挑选最好的参数
    # net_dropped.eval() 不启用dropout
    test_loss = 0
    correct = 0
    for data, target in val_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)
        correct += pred.eq(target).float().sum().item()
        
    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test_loss', update='append')
    viz.images(data.view(-1, 1, 28, 28)*0.3081+0.1307, win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))
        
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.299585
Train Epoch: 0 [20000/50000 (40%)]	Loss: 2.050581
Train Epoch: 0 [40000/50000 (80%)]	Loss: 1.401711

Test set: Average loss: 0.0051, Accuracy: 7266.0/10000 (73%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 1.020472
Train Epoch: 1 [20000/50000 (40%)]	Loss: 0.674887
Train Epoch: 1 [40000/50000 (80%)]	Loss: 0.568124

Test set: Average loss: 0.0024, Accuracy: 8711.0/10000 (87%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.479894
Train Epoch: 2 [20000/50000 (40%)]	Loss: 0.492326
Train Epoch: 2 [40000/50000 (80%)]	Loss: 0.381439

Test set: Average loss: 0.0019, Accuracy: 8906.0/10000 (89%)



In [8]:
#多次调参重复上面循环，最后用test_loader来验收结果，这个跑一次后就不能动了。
test_loss = 0
correct = 0
for data, target in test_loader:
    data = data.view(-1, 28 * 28)
    data, target = data.to(device), target.cuda()
    logits = net(data)
    test_loss += criteon(logits, target).item()

    pred = logits.data.max(1)[1]
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0018, Accuracy: 8971/10000 (90%)



[WinError 10054] 远程主机强迫关闭了一个现有的连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目标计算机积极拒绝，无法连接。
[WinError 10061] 由于目